## Caida Normal Caida Fuerte (CNCF)
##### Los precios caigan y toquen PM40, antes deben de haber caido 3 - 5 puntos (Caida Normal)
##### si los precios caen mas de 5 puntos (Caida fuerte)
##### Vela martillo indica que esta parando la caida
##### Trazo una linea de tendencia siguiendo la caida, entra cuando rompe la tendencia

In [ ]:
import pandas as pd
import numpy as np
#import time
#import matplotlib.pyplot as plt
import os
import talib as ta
from scipy import stats
from scipy.signal import argrelextrema
from datetime import date, datetime
#from dateutil.relativedelta import relativedelta
from bokeh.plotting import figure, show, column
from sklearn.linear_model import LinearRegression
from backtesting import Backtest, Strategy

In [ ]:
df_h=pd.read_csv('notebooks/data/dataxh.txt',sep='\t')
tickers = [
    'SPY',
    'META',
    'AAPL'
]

#    'SPY',
#    'META',
#    'AAPL',
#    'AMZN',
#    'NFLX',
#    'MRNA',
#    'TSLA',
#    'TNA',
#    'GLD',
#    'SLV',
#    'USO',
#    'BAC',
#    'CVX',
#    'XOM',
#    'QQQ'

In [8]:
df = pd.DataFrame()
for ticker in tickers:
    company = df_h.query("companyName==@ticker").copy()
    company.sort_values(by=['datetime'])
    #company = yf.download(ticker, start = ini2_str, end = today_str, interval='60m')
    #company.columns = [company.columns[0][0], company.columns[1][0], company.columns[2][0], company.columns[3][0], company.columns[4][0]]
    #company.rename(columns={'Datetime':'Gmt time'}, inplace = True)
    #company['datetime'] = pd.to_datetime (company.index)
    #company['companyName'] = ticker
    company['SMA20'] = company['Close'].rolling(20).mean()
    company.dropna(inplace=False)
    company['SMA40'] = company['Close'].rolling(40).mean()
    company.dropna(inplace=False)

    #EVALUACION DE VELAS ALCISTAS
    company["CDLHAMMER"] = ta.CDLHAMMER(company['Open'], company['High'], company['Low'], company['Close'])
    company["CDLINVERTEDHAMMER"] = ta.CDLINVERTEDHAMMER(company['Open'], company['High'], company['Low'], company['Close']) 
    company["CDLMORNINGSTAR"] = ta.CDLMORNINGSTAR(company['Open'], company['High'], company['Low'], company['Close'])
    
    company["CDLHARAMI"] = ta.CDLHARAMI(company['Open'], company['High'], company['Low'], company['Close'])   
    company["CDLENGULFING"] = ta.CDLENGULFING(company['Open'], company['High'], company['Low'], company['Close']) 

    company["CDLMORNINGDOJISTAR"] = ta.CDLMORNINGDOJISTAR(company['Open'], company['High'], company['Low'], company['Close']) 
    company["CDLTAKURI"] = ta.CDLTAKURI(company['Open'], company['High'], company['Low'], company['Close']) 
    company["CDLDRAGONFLYDOJI"] = ta.CDLDRAGONFLYDOJI(company['Open'], company['High'], company['Low'], company['Close']) 
    company["CDLMARUBOZU"] = ta.CDLMARUBOZU(company['Open'], company['High'], company['Low'], company['Close'])     

    company["CDL3BLACKCROWS"] = ta.CDL3BLACKCROWS(company['Open'], company['High'], company['Low'], company['Close'])
    company["CDL2CROWS"] = ta.CDL2CROWS(company['Open'], company['High'], company['Low'], company['Close'])
    company["CDLIDENTICAL3CROWS"] = ta.CDLIDENTICAL3CROWS(company['Open'], company['High'], company['Low'], company['Close'])

    company["CDL3WHITESOLDIERS"] = ta.CDL3WHITESOLDIERS(company['Open'], company['High'], company['Low'], company['Close'])



    #df["cdlpiercing"] = ta.CDLPIERCING(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    #df["cdlclosingmarubozu"] = ta.CDLCLOSINGMARUBOZU(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    #df["cdlmarubozu"] = ta.CDLMARUBOZU(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    #df["cdl3whitesoldiers"] = ta.CDL3WHITESOLDIERS(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    #df["cdlharami"] = ta.CDLHARAMI(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    #df["cdlharamicross"] = ta.CDLHARAMICROSS(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])    
    #df["cdlladderbottom"] = ta.CDLLADDERBOTTOM(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])

     
    
    df = pd.concat([df, company],ignore_index=True)

In [9]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [10]:
def tipo_vela(df):
    if (df['CDLHAMMER']>0):
        return 'Hammer'
    elif (df['CDLINVERTEDHAMMER']>0):
        return 'InvHammer'
    elif (df['CDLMORNINGSTAR']>0):
        return 'MorningStar'    
    elif (df['CDLHARAMI']>0):
        return 'Harami'
    elif (df['CDLENGULFING']>0):
        return 'Engulfing'        
    elif (df['CDLMORNINGDOJISTAR']>0):
        return 'MorningDojiStar'
    elif (df['CDLTAKURI']>0):
        return 'Takuri '
    elif (df['CDLDRAGONFLYDOJI']>0):
        return 'DragonflyDoji'
    elif (df['CDLMARUBOZU']>0):
        return 'Marubozu'
    elif (df['CDL3BLACKCROWS']>0):
        return '3BlackCrows'
    elif (df['CDL2CROWS']>0):
        return '2Crows'
    elif (df['CDLIDENTICAL3CROWS']>0):
        return 'Identical3Crows'
    elif (df['CDL3WHITESOLDIERS']>0):
        return '3WhiteSoldiers'
df['tipo_vela'] = df.apply(tipo_vela, axis=1)


In [11]:
df = df.drop(['CDLHAMMER', 'CDLINVERTEDHAMMER','CDLMORNINGSTAR','CDLHARAMI','CDLENGULFING',
              'CDLMORNINGDOJISTAR','CDLTAKURI','CDLDRAGONFLYDOJI','CDLMARUBOZU','CDL3BLACKCROWS',
              'CDL2CROWS','CDLIDENTICAL3CROWS','CDL3WHITESOLDIERS'], axis=1)

In [12]:
df[df['tipo_vela'].isna()!=True]

,Close,High,Low,Open,Volume,datetime,companyName,SMA20,SMA40,tipo_vela
13,421.809998,421.980011,420.869995,421.894989,15350720,2023-06-01 19:30:00+00:00,SPY,NaN,NaN,Hammer
16,427.446503,427.690002,426.760010,427.559998,8755850,2023-06-02 15:30:00+00:00,SPY,NaN,NaN,Takuri
17,428.260010,428.385010,427.290009,427.440002,7488936,2023-06-02 16:30:00+00:00,SPY,NaN,NaN,Engulfing
22,428.899994,428.980011,427.782806,427.885010,14162419,2023-06-05 14:30:00+00:00,SPY,423.443947,NaN,Marubozu
28,427.421387,427.630005,425.989990,426.670013,10780330,2023-06-06 13:30:00+00:00,SPY,426.296899,NaN,Engulfing
...,...,...,...,...,...,...,...,...,...,...
10435,201.970001,202.220001,201.669998,201.880005,3552562,2025-05-22 17:30:00+00:00,AAPL,204.542045,207.365550,Engulfing
10448,198.550000,198.740000,198.020000,198.700000,133430,2025-05-27 07:00:00+00:00,AAPL,199.126885,203.046960,Hammer
10452,200.070000,200.093000,198.105000,198.170000,6847564,2025-05-27 11:00:00+00:00,AAPL,198.648510,202.077212,Marubozu
10459,200.360000,200.470000,200.110000,200.460000,88516,2025-05-27 18:00:00+00:00,AAPL,198.586755,200.706543,Takuri


## Detect Pivots/Fractals

In [13]:
ord=10
#for ticker in tickers:
#print("====", ticker)
#dfpl = df[(df.companyName==ticker)] #df.query("companyName=='AAPL'")
max_idx = argrelextrema(df['Close'].values, np.greater, order=ord)[0]
min_idx = argrelextrema(df['Close'].values, np.less, order=ord)[0]
#print(max_idx)
#print(min_idx)
#df['pivotHigh'] = np.nan
#df['pivotLow'] = np.nan
#df['isPivot'] = np.nan
#df['isPivot'] = np.nan
# Aplicar el cálculo solo a los índices en la lista
df.loc[max_idx, 'pivotHigh'] = df['High']+1e-3
df.loc[min_idx, 'pivotLow'] = df['Low']-(1e-3)
df.loc[max_idx, 'isPivot'] = 1
df.loc[min_idx, 'isPivot'] = 2

In [16]:
#CODIGO DE REVISION CARLOS
#dfpl = df.iloc[500:970]
#id: 479
dfpl = df.iloc[404:554]

p = figure(width=2500, height=500,
        title="CNCF",
        background_fill_color="#efefef",
        tooltips=[("Index", "@index"), ("Open", "@Open"), ("High","@High"), ("Low","@Low"), ("Close","@Close")]
        )
p.xaxis.major_label_orientation = 0.8 # radians
p.x_range.range_padding = 0.05
#p.xaxis.axis_line_join = "bevel" # radians
p.xaxis.axis_line_width = 2

p.segment("index", "High", "index","Low",  color="black", line_width=1, source=dfpl)

inc = dfpl.query("Close>Open")
dec = dfpl.query("Open>Close")

p.vbar(    
    x="index",
    width=0.6,
    bottom="Open",
    top="Close",
    fill_color="red",
    line_color="red",    
    source=dec   
)


p.vbar(    
    x="index",
    width=0.6,
    bottom="Open",
    top="Close",
    fill_color="green",
    line_color="green", 
    source=inc   
)


p.line(
    x="index", 
    y="SMA20", 
    color="#ffb81c",
    legend_label="SMA20",
    source=dfpl)

p.line(
    x="index", 
    y="SMA40", 
    color="red",
    legend_label="SMA40",
    source=dfpl)

p.scatter(x="index", y="pivotHigh", marker="circle", size=10,
           line_color="navy", fill_color="blue", alpha=0.5, legend_label="Cambio Tendencia Bajista", source=dfpl[(dfpl.isPivot==1)])

p.scatter(x="index", y="pivotLow", marker="circle", size=10,
           line_color="navy", fill_color="green", alpha=0.5, legend_label="Cambio Tendencia Alcista", source=dfpl[(dfpl.isPivot==2)])

show(p)

In [17]:
diffs = []
for i, row in df.iterrows():
    pivotHigh = row['pivotHigh']
    valor_actual = row['pivotLow']
    
    #if pivotHigh!=np.nan:  # solo aplicar la lógica cuando tipo sea 'X'
    if valor_actual>0:  # solo aplicar la lógica cuando tipo sea 'X'      
        valor_previo = df.loc[:i - 1, 'pivotHigh'][::-1].dropna()
        if not valor_previo.empty:
            ultimo_valor = valor_previo.iloc[0]
        else:
            ultimo_valor = np.nan  # o np.nan
    
        dif = ultimo_valor - valor_actual 
       
    else:
        dif = np.nan # o 0, o dejar el valor original si ya existía        
    diffs.append(dif)
df['dif_pivot'] = diffs

In [18]:
#df['dif_ant1'] = df['Close'].shift(1) - df['pivotLow']
#df['dif_ant2'] = df['Close'].shift(2) - df['pivotLow']
df['dif_ant1'] = df['Close'].shift(1) - df['Close']
df['dif_ant2'] = df['Close'].shift(2) - df['Close']
df[df['dif_pivot'].notna()]

,Close,High,Low,Open,Volume,datetime,companyName,SMA20,SMA40,tipo_vela,pivotHigh,pivotLow,isPivot,dif_pivot,dif_ant1,dif_ant2
38,426.559998,427.549988,426.140015,426.869995,7982012,2023-06-07 16:30:00+00:00,SPY,427.784158,NaN,None,NaN,426.139015,2.0,3.331987,0.299988,1.189117
74,434.489990,437.510010,433.589996,437.290009,18848504,2023-06-14 17:30:00+00:00,SPY,434.685779,431.631113,None,NaN,433.588996,2.0,-1.688002,2.790497,3.670105
125,431.429993,432.850006,431.190002,432.640015,16895156,2023-06-26 19:30:00+00:00,SPY,433.863991,435.604056,None,NaN,431.189002,2.0,12.711991,1.209015,0.684998
165,437.350006,437.850006,437.059998,437.730011,9701525,2023-07-06 14:30:00+00:00,SPY,442.536256,439.333952,None,NaN,437.058998,2.0,7.072007,0.369995,5.779999
180,438.390015,439.079987,437.584991,438.709900,6454870,2023-07-10 15:30:00+00:00,SPY,440.075380,440.898018,None,NaN,437.583991,2.0,5.057023,0.329987,0.879883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10273,190.613998,191.750000,190.139999,191.119995,3331629,2025-04-21 16:30:00+00:00,AAPL,195.981065,197.931585,None,NaN,190.138999,2.0,15.671998,0.516006,0.121002
10319,209.321198,209.720001,206.670502,209.300003,7770541,2025-04-30 13:30:00+00:00,AAPL,209.685893,207.138279,None,NaN,206.669502,2.0,-0.858504,1.918808,1.978806
10358,194.009995,195.360001,193.300003,195.320007,5113387,2025-05-07 17:30:00+00:00,AAPL,198.715885,204.220998,None,NaN,193.299003,2.0,19.901994,1.290009,0.860001
10410,206.619904,208.270004,204.259995,208.024994,15175611,2025-05-19 13:30:00+00:00,AAPL,211.063530,209.853514,None,NaN,204.258995,2.0,9.682008,4.570099,5.105103


##### Casos
###### Precio esta por debajo o cerca a PM40 y ademas haya caido mas de 3 puntos
###### Caida normal entre 3-5 puntos
###### Caida fuerte mas de 5 puntos

In [19]:
def cncf01(df):
    if ((df['isPivot']==2) & (df['dif_ant1']>=3) & (df['dif_ant1']<=5) & (df['SMA40']>df['SMA20'])):
        return 1
    elif ((df['isPivot']==2) & (df['dif_ant1']>5) & (df['SMA40']>df['SMA20'])):
        return 2
    else:
        return 0
df['cncf01'] = df.apply(cncf01, axis=1)

#### Detect price channel

In [20]:
def collect_channel(candle, backevalTrend, trendH): 
    localdf = dfpl.loc[candle-backevalTrend:candle] #tomar en cuenta el backcandles
    highs = localdf.loc[localdf.index.isin(trendH)].High.values
    idxhighs = localdf.loc[localdf.index.isin(trendH)].High.index
    if len(highs)>=2:       
        sl_highs, interc_highs, r_value_h, _, _ = stats.linregress(idxhighs,highs)       
        return(sl_highs, interc_highs, r_value_h**2)
    else:
        return(0,0,0)    

In [21]:
def backcandle(candle):
    
    trendH = []
    backeval = dfpl.index[0]
    val = 0
    
    dfpl['PM40_PM20'] = dfpl['SMA40']-dfpl['SMA20']

    if (dfpl.loc[candle,"PM40_PM20"]>0):
        idx_negativo = dfpl.loc[:candle, 'PM40_PM20'][::-1].lt(0).idxmax()
        backeval = idx_negativo - 4
        val = candle - backeval
    
    if val>75:
        backeval=candle-75

    #print ("backeval:", backeval)
        
    if (dfpl.loc[backeval,"PM40_PM20"]>0):
        idx_negativo = dfpl.loc[:backeval, 'PM40_PM20'][::-1].lt(0).idxmax()
        backeval = idx_negativo - 4

    if val>75:
        backeval=candle-75

    #ini = candle - backeval
    ini = backeval
    trendprev=dfpl.loc[ini:candle]
    trend = trendprev.reset_index(drop=False)

    idxhighs = trend.index.to_numpy().reshape(-1,1) # Convertir X a una matriz 2D
    highs = trend.High.values
    modelo1 = LinearRegression()
    modelo1.fit(idxhighs, highs)
    # Obtener la predicción de la línea de regresión
    Y_pred = modelo1.predict(idxhighs)
    # Calcular la distancia solo de los puntos que están POR ENCIMA de la línea
    trend['Distancia'] = trend["High"] - Y_pred  # Diferencia entre el valor real y la predicción
    # Filtrar los puntos que están por encima (donde Y > Y_pred, es decir, Distancia > 0)
    df_encima = trend[trend['Distancia'] > 0]
    puntos_mas_lejanos = df_encima.nlargest(3, 'Distancia')   #2 puntos mas lejanos
    trendH = puntos_mas_lejanos["index"].tolist()
    trendH.append(candle)   
    
    return trendH, backeval

### Detect break out

In [22]:
def trendChannel(row,candle, backeval, window, sl_highs, interc_highs):
    if (candle-backeval-window)<0:
        return np.nan
    ini=candle-backeval
    fin=candle+window
    trendcurrhigh = np.where(np.logical_or((row.index > fin),  (row.index <  ini)), np.nan, (sl_highs*row.index + interc_highs))
    return trendcurrhigh

In [23]:
def isBreakOut(candleEval,backcandles, window, start, candle):
    if (candleEval-backcandles-window)<0:
        return 0

    if candleEval==start:
        prev_idx = candleEval
    else:
        prev_idx = candleEval-1
        
    prev_high = dfpl.loc[prev_idx].High
    prev_low = dfpl.loc[prev_idx].Low
    prev_close = df.loc[prev_idx].Close
    
    curr_idx = candleEval
    curr_high = dfpl.loc[candleEval].High
    curr_low = dfpl.loc[candleEval].Low
    curr_close = dfpl.loc[candleEval].Close
    curr_open = dfpl.loc[candleEval].Open

    #trend_prevlow=dfpl.loc[prev_idx].trendcurrlow
    trend_prevhigh=dfpl.loc[prev_idx].trendcurrhigh

    #trend_currlow=dfpl.loc[candleEval].trendcurrlow
    trend_currhigh=dfpl.loc[candleEval].trendcurrhigh

    if ( prev_low < trend_prevhigh and
        prev_close > trend_prevhigh and
        curr_open > trend_currhigh and
        curr_close > trend_currhigh and
        candleEval>candle): #and r_sq_h > 0.9
        return 2

    elif (curr_open > trend_currhigh and
        curr_close > trend_currhigh and candleEval>candle):
        return 3
    
    else:
        return np.nan

In [24]:
def breakpointpos(x):
    if x['isBreakOutIni'] in (2,3):
        return x['Low']+3e-3
    elif x['isBreakOutIni']==1:
        return x['High']-3e-3
    else:
        return np.nan

In [27]:
## Evaluando casos
df_casos = pd.DataFrame()
caso = 0
ticker2 = ""
window=5
backeval=75
backevalTrend=0
dfpl = pd.DataFrame
trendH = []
trendL = []
cant = 0
for i, row in df.iterrows():
    if(df['cncf01'][i]>=1): #posibles casos 
        print("posible caso:",i, df['companyName'][i])
        cant = 0
        candle = i
        cnt = 0
        valiniHigh = df.loc[candle,"High"]
        #valiniLow = df.loc[candle,"Low"]
        ticker = df['companyName'][i]
        #Reinicio de casos por company
        if ticker2 != ticker:
            ticker2 = ticker
            caso = 0
        dfpl = df[(df.companyName==ticker)].loc[i-backeval:i+backeval].copy()
        #print("rowMin:", dfpl.index[0],", rowMax:", dfpl.index[-1])
                   
        h=1
        l=1

        cant = cant +1
        cantHorasTrend=0
        #print("opcionH:",h,"opcionL:",l)
        trendH, backevalTrend = backcandle(candle)
        #print ("-->trendH:",trendH)
        #print ("-->backevalTrend:",backevalTrend)    
        
        cantHorasTrend = (dfpl.loc[candle,"datetime"] - dfpl.loc[backevalTrend,"datetime"]).total_seconds() / 3600

        if (cantHorasTrend>48): #Verificacion de caida por lo menos 2 dias
           backevalTrend2 = candle - backevalTrend
           #print ("backevalTrend2", backevalTrend2)
                       
           #revision de trend negativo indica que esta cayendo, solo me quedo con estos casos
           sl_highs, interc_highs, r_sq_h = collect_channel(candle, backevalTrend2, trendH)
                   
    
           dfpl.loc[trendH, "trendH"] = 1
        
           
           #print("sl_highs:", sl_highs, ", sl_lows:", sl_lows, ", r_sq_h:", r_sq_h)
           if (sl_highs<=0): #solo tendencias a la baja and ind_vela>0 sl_lows<0 and r_sq_h>=0.7  and sl_highs<=-0.1 and ind_vela>0
               
               #crear linea de tendencia high y low
               dfpl['trendcurrhigh']  = trendChannel(dfpl,candle,backevalTrend2,window,sl_highs,interc_highs)                
          
               promVol = dfpl.loc[i-backeval:i]["Volume"].mean()                
               
               dfpl["bearishSlope"] = np.nan #pendiente bajista en vacio
               dfpl.loc[i, "bearishSlope"] = 1 #pendiente bajista solo al punto evaluado            
               dfpl["ind_posicion"] = 0
               dfpl.loc[(dfpl.index < i), 'ind_posicion'] = -1
               dfpl.loc[(dfpl.index > i), 'ind_posicion'] = 1
               dfpl["promVol"] = promVol
               dfpl["sl_highs"] = sl_highs
               dfpl["r_sq_h"] = r_sq_h
        
               start = dfpl.index[0]
               dfpl["isBreakOut"] = [isBreakOut(candleEval, backevalTrend2, window, start, candle) for candleEval in dfpl.index]
               # Solo me quedo con el primer BREAK OUT
               cnt = dfpl.query("isBreakOut in (2,3) and Close>Open").shape[0] 
               id=0
               id2=0
               if cnt>0:
                   id = dfpl.query("isBreakOut in (2,3) and Close>Open").index[0]                        
                   #dfpl['isBreakOutIni'] = np.where(id!=dfpl.index, np.nan, dfpl['isBreakOut'])                        
                   dfpl.loc[id,'isBreakOutIni'] = 1
                   dfpl['breakpointpos'] = dfpl.apply(lambda row: breakpointpos(row), axis=1)
                   cnt2 = dfpl.query("index>@id and isPivot==1").shape[0]
                   
                   if cnt2>0:
                       id2 = dfpl.query("index>@id and isPivot==1").index[0]
                       dfpl.loc[id2,'isBreakOutFinal'] = 1
                   else:
                       dfpl['isBreakOutFinal'] = np.nan                      
                   
               else:
                   dfpl['isBreakOutIni'] = np.nan                        
                   dfpl['breakpointpos'] = np.nan
        
               if cnt>0:
                   #INSERT CASO
                   caso=caso+1                            
                   dfpl["caso"] = caso
                   #print(i, df['companyName'][i])
                   if len(df_casos)<1:
                       df_casos = dfpl
                   else:
                       df_casos = pd.concat([df_casos, dfpl],ignore_index=False)

posible caso: 2017 SPY
posible caso: 2060 SPY
posible caso: 2242 SPY
posible caso: 2592 SPY
posible caso: 2822 SPY
posible caso: 3025 SPY
posible caso: 3098 SPY
posible caso: 3193 SPY
posible caso: 3229 SPY
posible caso: 3459 SPY
posible caso: 3823 META
posible caso: 3871 META
posible caso: 4032 META
posible caso: 4064 META
posible caso: 4385 META
posible caso: 4944 META
posible caso: 5031 META
posible caso: 5050 META
posible caso: 5072 META
posible caso: 5098 META
posible caso: 5199 META
posible caso: 5215 META
posible caso: 5670 META
posible caso: 5929 META
posible caso: 6089 META
posible caso: 6231 META
posible caso: 6269 META
posible caso: 6324 META
posible caso: 6340 META
posible caso: 6516 META
posible caso: 6602 META
posible caso: 6630 META
posible caso: 6683 META
posible caso: 6938 META
posible caso: 7451 AAPL
posible caso: 8161 AAPL
posible caso: 8839 AAPL
posible caso: 9046 AAPL
posible caso: 9186 AAPL
posible caso: 9211 AAPL
posible caso: 9801 AAPL


In [28]:
df_casos[(df_casos['caso']==1) & (df_casos['ind_posicion']==0)]

,Close,High,Low,Open,Volume,datetime,companyName,SMA20,SMA40,tipo_vela,...,bearishSlope,ind_posicion,promVol,sl_highs,r_sq_h,isBreakOut,isBreakOutIni,breakpointpos,isBreakOutFinal,caso
2017,538.340027,543.309998,538.130005,543.130005,10394986,2024-07-25 19:30:00+00:00,SPY,546.738361,549.419061,None,...,1.0,0,6.169620e+06,-0.731273,0.981004,NaN,NaN,NaN,NaN,1
4032,293.920013,298.345001,293.269989,295.700012,5566544,2023-09-21 13:30:00+00:00,META,302.620804,304.114916,None,...,1.0,0,2.416075e+06,-1.026390,0.815050,NaN,NaN,NaN,NaN,1
8161,183.315002,184.169998,180.050003,180.449997,35096943,2024-02-02 14:30:00+00:00,AAPL,186.268195,189.311170,None,...,1.0,0,6.687172e+06,-0.298133,0.893876,NaN,NaN,NaN,NaN,1


In [29]:
df_casos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3171 entries, 1942 to 9121
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   Close            3171 non-null   float64            
 1   High             3171 non-null   float64            
 2   Low              3171 non-null   float64            
 3   Open             3171 non-null   float64            
 4   Volume           3171 non-null   int64              
 5   datetime         3171 non-null   datetime64[ns, UTC]
 6   companyName      3171 non-null   object             
 7   SMA20            3171 non-null   float64            
 8   SMA40            3171 non-null   float64            
 9   tipo_vela        461 non-null    object             
 10  pivotHigh        100 non-null    float64            
 11  pivotLow         100 non-null    float64            
 12  isPivot          200 non-null    float64            
 13  dif_pivot        100

In [ ]:
#descarga de informacion, estrategia: RUPTURA DEL CANAL BAJISTA
path='notebooks/data/cncf_h.txt'
# check whether the file exists
if os.path.exists(path):
    # delete the file
    os.remove(path)
else:
    # if the file does not exist.
    print("File does not exists. File needs to be created.")

#export DataFrame to text file
with open(path, 'a') as f:
    #df_string = appl_hor3.to_string(header=True, index=False, sep ='\t')
    df_casos.to_csv(path, header=True, index=None, sep='\t', mode='w')
    #f.write(df_string)

File does not exists. File needs to be created.


In [31]:
df_casos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3171 entries, 1942 to 9121
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   Close            3171 non-null   float64            
 1   High             3171 non-null   float64            
 2   Low              3171 non-null   float64            
 3   Open             3171 non-null   float64            
 4   Volume           3171 non-null   int64              
 5   datetime         3171 non-null   datetime64[ns, UTC]
 6   companyName      3171 non-null   object             
 7   SMA20            3171 non-null   float64            
 8   SMA40            3171 non-null   float64            
 9   tipo_vela        461 non-null    object             
 10  pivotHigh        100 non-null    float64            
 11  pivotLow         100 non-null    float64            
 12  isPivot          200 non-null    float64            
 13  dif_pivot        100

### BACKTESTING
##### 
Usando paquete backtesting.py

In [33]:
df_casos2 = df_casos.query("isBreakOutIni==1 or isBreakOutFinal==1").copy()

In [34]:
# =============== BACKTESTING ================= #
# obteniendo dataframe backtesting
df_backTesting = pd.merge(df, df_casos2[['datetime','companyName', 'isBreakOutIni','isBreakOutFinal','caso']], on = ['companyName','datetime'], how='left')
# datos de trade se asigna una vela anterior debido a que backtesting.py toma entra como la siguiente vela
df_backTesting['isBreakOutIni'] = df_backTesting['isBreakOutIni'].shift(-1)
df_backTesting['isBreakOutFinal'] = df_backTesting['isBreakOutFinal'].shift(-1)
#modificamos indice con el compo datetime
df_backTesting.set_index('datetime', inplace=True)

In [35]:
#Strategia
class strategyRupturaCanal(Strategy):    
    def init(self):        
        self.breakout_entry = self.I(lambda x: x, self.data.isBreakOutIni) #Indica la señal entrada al backtesting
        self.breakout_exit = self.I(lambda x: x, self.data.isBreakOutFinal) #Indica la señal salida al backtesting

    def next(self):       
        #print(self.data)        
        if self.breakout_entry[-1] == 1:
            self.buy()  #COMPRA
        elif self.breakout_exit[-1] == 1 and self.position:
            self.position.close() #CIERRE

In [36]:
# Por cada Ticket
my_stats = {}
estadisticas = pd.DataFrame()          # lista para ticker
tradesprev = pd.DataFrame()     # lista para cada caso por ticker
for ticker in df_backTesting['companyName'].unique():
    ticker_data = df_backTesting[df_backTesting['companyName'] == ticker].copy() #Filtro por cada ticker
    #ticker_data=ticker_data.sort_values(by='caso')
    #print("\n" + "="*50 + "\n")
    bt = Backtest(ticker_data, strategyRupturaCanal, cash=10_000)
    #stats = bt.run()
    #my_stats[f"stats_BreakOUT_{ticker}"] = bt.run()
    my_stats = bt.run()
    listTrades = my_stats['_trades']
    tradesdf = pd.DataFrame(listTrades) #trades
    tradesdf['Ticker'] = ticker
    my_stast_filter=pd.DataFrame([my_stats])
    if tradesprev.shape[0]==0:
        tradesprev = tradesdf.copy()
        estadisticas=my_stast_filter.copy()
    else:
        tradesprev = pd.concat([tradesprev, tradesdf],ignore_index=True)
        estadisticas = pd.concat([estadisticas, my_stast_filter],ignore_index=True)
    # Mostramos
    print(f"Resultados para {ticker}:")
    print(stats)   
    #plt.title(f"Backtest -  {ticker}")
    bt.plot(resample=False, filename=f"plot_BreakOUT_{ticker}")

Backtest.run:   0%|          | 0/1469 [00:00<?, ?bar/s]

Resultados para SPY:
<module 'scipy.stats' from 'C:\\Users\\LINDER\\anaconda3\\Lib\\site-packages\\scipy\\stats\\__init__.py'>


C:\Users\LINDER\AppData\Local\Temp\ipykernel_8908\1326577465.py:27: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot(resample=False, filename=f"plot_BreakOUT_{ticker}")


Backtest.run:   0%|          | 0/2906 [00:00<?, ?bar/s]

Resultados para META:
<module 'scipy.stats' from 'C:\\Users\\LINDER\\anaconda3\\Lib\\site-packages\\scipy\\stats\\__init__.py'>


C:\Users\LINDER\AppData\Local\Temp\ipykernel_8908\1326577465.py:27: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot(resample=False, filename=f"plot_BreakOUT_{ticker}")


Backtest.run:   0%|          | 0/2286 [00:00<?, ?bar/s]

Resultados para AAPL:
<module 'scipy.stats' from 'C:\\Users\\LINDER\\anaconda3\\Lib\\site-packages\\scipy\\stats\\__init__.py'>


C:\Users\LINDER\AppData\Local\Temp\ipykernel_8908\1326577465.py:27: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot(resample=False, filename=f"plot_BreakOUT_{ticker}")


In [37]:
ini = df_casos2[(df_casos2['isBreakOutIni']==1)].copy()
df_casos3_ini=ini.rename(columns={'Open': 'EntryPrice', 'datetime': 'EntryTime', 'companyName': 'Ticker'})[['EntryPrice','Ticker','caso','EntryTime']]
final = df_casos2[(df_casos2['isBreakOutFinal']==1)].copy()
df_casos3_final=final.rename(columns={'Open': 'ExitPrice', 'datetime': 'ExitTime', 'companyName': 'Ticker'})[['ExitPrice','Ticker','caso','ExitTime']]

In [38]:
df_casos3=df_casos3_ini.merge(df_casos3_final,on=['Ticker','caso'],how='outer')

In [39]:
trades = pd.merge(df_casos3, tradesprev, on = ['EntryTime','Ticker'], how='left')

In [40]:
def exitPrice(x):
    if pd.isna(x['ExitPrice_x']):
        return x['ExitPrice_y']
    else:
        return x['ExitPrice_x']
trades['ExitPrice'] = trades.apply(lambda row: exitPrice(row), axis=1)

In [41]:
trades = trades.rename(columns={'EntryPrice_x': 'EntryPrice'})
trades = trades.drop(['EntryPrice_y','ExitPrice_x', 'ExitPrice_y'], axis=1)

In [42]:
def exitTime(x):
    if pd.isna(x['ExitTime_x']):
        return x['ExitTime_y']
    else:
        return x['ExitTime_x']
trades['ExitTime'] = trades.apply(lambda row: exitTime(row), axis=1)

In [43]:
def duration(x):
    if pd.isna(x['Duration']):
        if pd.isna(x['ExitTime']):
            return pd.NaT
        else:
            return x['ExitTime']-x['EntryTime']
    else:
        return x['Duration']
trades['Duration'] = trades.apply(lambda row: duration(row), axis=1)

In [44]:
trades = trades.drop(['ExitTime_x', 'ExitTime_y'], axis=1)

In [45]:
trades=trades[['Ticker','EntryTime','ExitTime','EntryPrice','ExitPrice','Duration','Size','EntryBar','ExitBar','ReturnPct','caso']]

In [46]:
trades

,Ticker,EntryTime,ExitTime,EntryPrice,ExitPrice,Duration,Size,EntryBar,ExitBar,ReturnPct,caso
0,AAPL,2024-02-02 16:30:00+00:00,2024-02-07 14:30:00+00:00,187.009995,190.639999,4 days 22:00:00,NaN,NaN,NaN,NaN,1
1,AAPL,2024-08-06 14:30:00+00:00,2024-08-16 16:30:00+00:00,204.979996,226.350006,10 days 02:00:00,48.0,2068.0,2126.0,0.104254,2
2,META,2023-09-21 16:30:00+00:00,2023-09-29 13:30:00+00:00,297.420013,307.380005,7 days 21:00:00,NaN,NaN,NaN,NaN,1
3,META,2023-09-28 13:30:00+00:00,2023-09-29 13:30:00+00:00,298.934998,307.380005,1 days 00:00:00,NaN,NaN,NaN,NaN,2
4,META,2023-12-04 18:30:00+00:00,2023-12-08 20:30:00+00:00,317.549988,332.341705,4 days 02:00:00,31.0,900.0,930.0,0.046581,3
5,META,2024-04-01 16:30:00+00:00,2024-04-05 15:30:00+00:00,489.929993,525.469971,3 days 23:00:00,21.0,1459.0,1486.0,0.072541,4
6,META,2024-04-18 13:30:00+00:00,2024-04-18 14:30:00+00:00,499.799988,506.829987,0 days 01:00:00,22.0,1547.0,1548.0,0.014066,5
7,META,2024-04-25 18:30:00+00:00,2024-04-26 19:30:00+00:00,437.429993,442.294006,1 days 01:00:00,25.0,1587.0,1595.0,0.011120,6
8,META,2024-05-24 14:30:00+00:00,2024-05-28 19:30:00+00:00,475.321899,476.599915,4 days 05:00:00,24.0,1730.0,1742.0,0.002689,7
9,META,2024-10-21 19:30:00+00:00,2024-10-22 19:30:00+00:00,573.479980,580.969971,1 days 00:00:00,20.0,2445.0,2452.0,0.013061,8


In [ ]:
#path = r'D:\PRUEBAS'
path = 'notebooks/data/backtesting'
path_trades = os.path.join(path, 'trades_cncf.txt')
path_estadisticas = os.path.join(path, 'estadisticas_cncf.txt')

for file_path in [path_trades, path_estadisticas]:
    if os.path.exists(file_path):
        os.remove(file_path)
    else:
        print(f"File does not exist: {file_path}. It will be created.")

trades.to_csv(path_trades, header=True, index=False, sep='\t', mode='w')
estadisticas.to_csv(path_estadisticas, header=True, index=False, sep='\t', mode='w')

File does not exist: D:\PRUEBAS\trades_cncf.txt. It will be created.
File does not exist: D:\PRUEBAS\estadisticas_cncf.txt. It will be created.
